In [ ]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 5 QUESTION 3

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import torch
import time
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
housing = pd.read_csv('Housing.csv')
housing

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [3]:
#Choosing the values of the columns as per the question
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df_new = housing[num_vars]
df_new

,price,area,bedrooms,bathrooms,stories,parking
0,13300000,7420,4,2,3,2
1,12250000,8960,4,4,4,3
2,12250000,9960,3,2,2,2
3,12215000,7500,4,2,2,3
4,11410000,7420,4,1,2,2
...,...,...,...,...,...,...
540,1820000,3000,2,1,1,2
541,1767150,2400,3,1,1,0
542,1750000,3620,2,1,1,0
543,1750000,2910,3,1,1,0


In [4]:
df_new.shape

(545, 6)

In [5]:
#Scaling the data
scaler = MinMaxScaler()
df_new[num_vars] = scaler.fit_transform(df_new[num_vars])
df_new.head()

C:\Users\Shruthi\AppData\Local\Temp\ipykernel_19956\2943118898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[num_vars] = scaler.fit_transform(df_new[num_vars])


,price,area,bedrooms,bathrooms,stories,parking
0,1.000000,0.396564,0.6,0.333333,0.666667,0.666667
1,0.909091,0.502405,0.6,1.000000,1.000000,1.000000
2,0.909091,0.571134,0.4,0.333333,0.333333,0.666667
3,0.906061,0.402062,0.6,0.333333,0.333333,1.000000
4,0.836364,0.396564,0.6,0.000000,0.333333,0.666667


In [6]:
#Splitting the dataset into parameters and labels
X = df_new.iloc[:, 1:6].values
Y = df_new.iloc[:, 0].values

In [7]:
#Converting into tensors
X = torch.tensor(X)
Y = torch.tensor(Y)

In [8]:
#Splitting the test and train ratio as 20-80 
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)
#Shuffling the indices of the original dataset so that the model can be trained more effectively
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [9]:
#Splitting the data into train and test and into parameters and target/labels
train_X = X[train_indices]
train_Y = Y[train_indices]
#Here we add an extra dimension, to make the labels from (len,) to (len,1)
train_Y = train_Y.unsqueeze(1)

val_X = X[val_indices]
val_Y = Y[val_indices]
#Here we add an extra dimension, to make the labels from (len,) to (len,1)
val_Y = val_Y.unsqueeze(1)
print(train_X.shape)
print(train_Y.shape)
print(val_X.shape)
print(val_Y.shape)

torch.Size([436, 5])
torch.Size([436, 1])
torch.Size([109, 5])
torch.Size([109, 1])


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

# The first model, with only one hidden block

model = nn.Sequential(
            nn.Linear(5,2),
            nn.ReLU(), 
            nn.LogSoftmax(dim=1))
model = model.float()
learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

out = model(train_X.float())


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0
#Defining the training loop using the model defined above with the validation and the training and printing selected epochs, 
# and pringting out the training time
for epoch in range(n_epochs+1):
        training_start_time = time.time()
        out = model(train_X.float())
        loss = loss_fn(out,train_Y.float())
        
        out_v= model(val_X.float()) # <1>
        val_loss = loss_fn(out_v, val_Y.float())
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 20 ==0:
            print("Epoch: %d, Training Loss: %f, Validation Loss: %f" % (epoch, float(loss), float(val_loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_Y.shape[0]
        correct += int((predicted == train_Y).sum())
        
print("Accuracy: %f" % (correct / total))
print('Training finished, took {:.2f}s'.format(time.time() - training_start_time))   

C:\Users\Shruthi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([436, 1])) that is different to the input size (torch.Size([436, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Shruthi\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([109, 1])) that is different to the input size (torch.Size([109, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, Training Loss: 0.972274, Validation Loss: 0.916092
Epoch: 20, Training Loss: 0.963600, Validation Loss: 0.907279
Epoch: 40, Training Loss: 0.958450, Validation Loss: 0.902014
Epoch: 60, Training Loss: 0.955276, Validation Loss: 0.898763
Epoch: 80, Training Loss: 0.953241, Validation Loss: 0.896698
Epoch: 100, Training Loss: 0.951895, Validation Loss: 0.895352
Epoch: 120, Training Loss: 0.950976, Validation Loss: 0.894455
Epoch: 140, Training Loss: 0.950327, Validation Loss: 0.893837
Epoch: 160, Training Loss: 0.949861, Validation Loss: 0.893402
Epoch: 180, Training Loss: 0.949519, Validation Loss: 0.893089
Epoch: 200, Training Loss: 0.949266, Validation Loss: 0.892862
Accuracy: 12.000000
Training finished, took 0.00s


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim

# The second model, with three hidden blocks

model2 = nn.Sequential(
             nn.Linear(5,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,2),
            nn.ReLU(),    
            nn.LogSoftmax(dim=1))
model = model.float()
learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

out = model2(train_X.float())


loss_fn = nn.MSELoss()

n_epochs = 200
correct = 0
total = 0
#Defining the training loop using the model defined above with the validation and the training and printing selected epochs, 
# and pringting out the training time
for epoch in range(n_epochs+1):
        training_start_time = time.time()
        out = model2(train_X.float())
        loss = loss_fn(out,train_Y.float())
        
        out_v= model2(val_X.float()) # <1>
        val_loss = loss_fn(out_v, val_Y.float())
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch==0 or epoch % 20 ==0:
            print("Epoch: %d, Training Loss: %f, Validation Loss: %f" % (epoch, float(loss), float(val_loss)))
        _, predicted = torch.max(out, dim=1)
        total += val_Y.shape[0]
        correct += int((predicted == train_Y).sum())
        
print("Accuracy: %f" % (correct / total))
print('Training finished, took {:.2f}s'.format(time.time() - training_start_time))   

Epoch: 0, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 20, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 40, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 60, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 80, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 100, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 120, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 140, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 160, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 180, Training Loss: 1.077067, Validation Loss: 1.019441
Epoch: 200, Training Loss: 1.077067, Validation Loss: 1.019441
Accuracy: 12.000000
Training finished, took 0.00s
